In [ ]:
# default_exp diffquant_utils

In [ ]:
#export
def get_condpairname(condpair):
    return f"{condpair[0]}_VS_{condpair[1]}"

In [ ]:
#export
def get_middle_elem(sorted_list):
    nvals = len(sorted_list)
    if nvals==1:
        return sorted_list[0]
    middle_idx = nvals//2
    if nvals%2==1:
        return sorted_list[middle_idx]
    return 0.5* (sorted_list[middle_idx] + sorted_list[middle_idx-1])

In [ ]:
#export
import numpy as np
def get_nonna_array(array_w_nas):
    res = []
    isnan_arr = np.isnan(array_w_nas)

    for idx in range(len(array_w_nas)):
        sub_res = []
        sub_array = array_w_nas[idx]
        na_array = isnan_arr[idx]
        for idx2 in range(len(sub_array)):
            if not na_array[idx2]:
               sub_res.append(sub_array[idx2])
        res.append(np.array(sub_res))
    return np.array(res)

In [ ]:
#export
import numpy as np
def get_non_nas_from_pd_df(df):
    # vals = df.values
    # result_dict = dict()
    # pep_names = df.index.values
    # for pep_name, sub_vals in zip(pep_names, vals):
    #     result_dict[pep_name] = sub_vals[~np.isnan(sub_vals)]
    # return result_dict
    return {
        pep_name: sub_vals[~np.isnan(sub_vals)] for pep_name, sub_vals in
        zip( df.index.values, df.values)
    }